In [1]:
import pandas as pd
import numpy as np
#import scipy as sp
import datetime
import sklearn
from sklearn.model_selection import train_test_split

#%matplotlib inline

In [2]:
def oneHotEncoding(df, columna):
  
    one_hot = pd.get_dummies(df[columna])
    df.drop(columna, axis=1, inplace=True)
    df = pd.concat([df, one_hot], axis=1)
    return df

# Cargo CSV

In [3]:
test_final = pd.read_csv('../data/test_final_100k.csv')

In [ ]:
postulantes_educacion = pd.read_csv('../data/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')
postulantes_educacion = postulantes_educacion.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_1_postulantes_educacion.csv'))
postulantes_educacion = postulantes_educacion.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_1_postulantes_educacion.csv'))
postulantes_educacion = postulantes_educacion.reset_index().drop(columns='index')

In [4]:
postulantes_genero_y_edad = pd.read_csv('../data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_2_postulantes_genero_y_edad.csv'))
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop(columns='index')

In [ ]:
vistas = pd.read_csv('../data/datos_navent_fiuba/fiuba_3_vistas.csv')
vistas = vistas.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_3_vistas.csv'))
vistas = vistas.reset_index().drop(columns='index')

In [5]:
postulaciones = pd.read_csv('../data/datos_navent_fiuba/fiuba_4_postulaciones.csv')
postulaciones = postulaciones.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_4_postulaciones.csv'))
# no tiene postulaciones -> pd.read_csv('data/fiuba_desde_15_abril/')
postulaciones = postulaciones.reset_index().drop(columns='index')

In [ ]:
avisos_online = pd.read_csv('../data/datos_navent_fiuba/fiuba_5_avisos_online.csv')
avisos_online = avisos_online.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_5_avisos_online.csv'))
# no tiene avisos online pd.read_csv('data/fiuba_desde_15_abril/')
avisos_online = avisos_online.reset_index().drop(columns='index')

In [ ]:
avisos_detalle = pd.read_csv('../data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_hasta_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_desde_15_abril/fiuba_6_avisos_detalle.csv'))
avisos_detalle = avisos_detalle.append(pd.read_csv('../data/fiuba_6_avisos_detalle_missing_nivel_laboral.csv'))
avisos_detalle = avisos_detalle.reset_index().drop(columns='index')

# Preproceso datos

### Limpio el DF *postulantes_educacion*

In [ ]:
postulantes_educacion.head()

In [ ]:
postulantes_educacion.shape

In [ ]:
postulantes_educacion.isnull().sum()

In [ ]:
postulantes_educacion.isna().sum()

In [ ]:
postulantes_educacion[(postulantes_educacion.duplicated()) & (postulantes_educacion['idpostulante'] == 'NdJl')]

In [ ]:
postulantes_educacion = postulantes_educacion.drop_duplicates(keep='last')

In [ ]:
postulantes_educacion = postulantes_educacion.reset_index().drop(columns='index')

### Limpio el DF *postulantes_genero_y_edad*

In [6]:
postulantes_genero_y_edad.head()

,idpostulante,fechanacimiento,sexo
0,NM5M,1970-12-03,FEM
1,5awk,1962-12-04,FEM
2,ZaO5,1978-08-10,FEM
3,NdJl,1969-05-09,MASC
4,eo2p,1981-02-16,MASC


In [7]:
postulantes_genero_y_edad.shape

(780020, 3)

In [8]:
postulantes_genero_y_edad.isna().sum()

idpostulante           0
fechanacimiento    34273
sexo                   0
dtype: int64

In [9]:
postulantes_genero_y_edad.isnull().sum()

idpostulante           0
fechanacimiento    34273
sexo                   0
dtype: int64

Luego veremos que hacer con los ids sin fecha de nacimiento, por el momento los dejamos.

In [10]:
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates()

In [11]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zP6MJa']

,idpostulante,fechanacimiento,sexo
168364,0zP6MJa,1985-08-16,FEM
525452,0zP6MJa,1989-08-16,FEM


In [12]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zPvY49']

,idpostulante,fechanacimiento,sexo
485636,0zPvY49,1991-03-26,MASC
678829,0zPvY49,1991-03-26,FEM


In [13]:
#Dejo la ultima aparicion del duplicado, supongo que es el update más reciente
postulantes_genero_y_edad = postulantes_genero_y_edad.drop_duplicates(subset=['idpostulante'],keep='last')

In [14]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zP6MJa']

,idpostulante,fechanacimiento,sexo
525452,0zP6MJa,1989-08-16,FEM


In [15]:
postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'] == '0zPvY49']

,idpostulante,fechanacimiento,sexo
678829,0zPvY49,1991-03-26,FEM


In [16]:
postulantes_genero_y_edad['sexo'].value_counts()

FEM           251205
MASC          227853
NO_DECLARA     25342
0.0                7
Name: sexo, dtype: int64

Al igual que con la fecha de nacimiento, por el momento, dejamos los ids con sexo 0.0

In [17]:
postulantes_genero_y_edad = postulantes_genero_y_edad.reset_index().drop(columns='index')

### Limpio el DF *vistas*

In [ ]:
vistas.head()

In [ ]:
vistas.shape

In [ ]:
vistas.isna().sum()

In [ ]:
vistas.isnull().sum()

In [ ]:
vistas = vistas.drop_duplicates(keep='last')

In [ ]:
vistas = vistas.reset_index().drop(columns='index')

### Limpio el DF *postulaciones*

In [18]:
postulaciones.head()

,idaviso,idpostulante,fechapostulacion
0,1112257047,NM5M,2018-01-15 16:22:34
1,1111920714,NM5M,2018-02-06 09:04:50
2,1112346945,NM5M,2018-02-22 09:04:47
3,1112345547,NM5M,2018-02-22 09:04:59
4,1112237522,5awk,2018-01-25 18:55:03


In [19]:
postulaciones.shape

(8311264, 3)

In [20]:
postulaciones.isna().sum()

idaviso             0
idpostulante        0
fechapostulacion    0
dtype: int64

In [21]:
postulaciones.isnull().sum()

idaviso             0
idpostulante        0
fechapostulacion    0
dtype: int64

In [22]:
postulaciones = postulaciones.drop_duplicates(keep='last')

In [23]:
postulaciones = postulaciones.reset_index().drop(columns='index')

### Limpio el DF *avisos_online*

In [ ]:
avisos_online.head()

In [ ]:
avisos_online.shape

In [ ]:
avisos_online.isna().sum()

In [ ]:
avisos_online.isnull().sum()

In [ ]:
avisos_online = avisos_online.drop_duplicates(keep='last')

In [ ]:
avisos_online = avisos_online.reset_index().drop(columns='index')

### Limpio el DF *avisos_detalle*

In [ ]:
avisos_detalle.head()

In [ ]:
avisos_detalle.shape

In [ ]:
avisos_detalle['idaviso'].duplicated().sum()

In [ ]:
avisos_detalle = avisos_detalle.drop_duplicates(subset=['idaviso'],keep='last')

In [ ]:
avisos_detalle = avisos_detalle.reset_index().drop(columns='index')

# Preprocesamiento

In [24]:
# Agrego la edad de los postulantes
postulantes_genero_y_edad['fechanacimiento'] = pd.to_datetime(postulantes_genero_y_edad['fechanacimiento'],errors='coerce')
postulantes_genero_y_edad['edad'] = datetime.datetime.now().year - postulantes_genero_y_edad['fechanacimiento'].dt.year

In [25]:
postulantes_genero_y_edad.head()

,idpostulante,fechanacimiento,sexo,edad
0,NM5M,1970-12-03,FEM,48.0
1,5awk,1962-12-04,FEM,56.0
2,ZaO5,1978-08-10,FEM,40.0
3,NdJl,1969-05-09,MASC,49.0
4,eo2p,1981-02-16,MASC,37.0


In [26]:
# Despues veo que hago con los que no tienen edad
postulantes_genero_y_edad.isna().sum()

idpostulante           0
fechanacimiento    26096
sexo                   0
edad               26096
dtype: int64

In [27]:
# En un principio los relleno con la media
int(postulantes_genero_y_edad['edad'].mean())

31

In [28]:
# Separo gente postulada de no postulada
# Tupla (idpostulante,idaviso)
#tupla_postulada = postulaciones.drop(columns='fechapostulacion')

In [29]:
# Separa la gente postulada, luego los busco en el DF que junta todos los postulantes
# y me quedo con los opuestos, los no postulados.
# esto me va a servir para samplear
# GENTE NO POSTULADA
gente_no_postulada = postulantes_genero_y_edad[-postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_no_postulada = gente_no_postulada.reset_index(drop=True)

In [30]:
gente_no_postulada['sexo'].value_counts()

FEM           71992
MASC          67819
NO_DECLARA    15707
0.0               6
Name: sexo, dtype: int64

In [31]:
# Pocos datos con sexo invalido, los desecho ya que son los no postulados
gente_no_postulada = gente_no_postulada.drop(gente_no_postulada[gente_no_postulada['sexo'] == '0.0'].index)

In [32]:
# GENTE POSTULADA
gente_postulada = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].\
                                               isin(postulaciones['idpostulante'].value_counts().index)]
gente_postulada = gente_postulada.reset_index(drop=True)

In [33]:
gente_postulada['sexo'].value_counts()

FEM           179213
MASC          160034
NO_DECLARA      9635
0.0                1
Name: sexo, dtype: int64

In [34]:
# Reemplazo el sexo invalido por NO_DECLARA
gente_postulada.loc[gente_postulada['sexo'] == '0.0','sexo'] = 'NO_DECLARA'

# ........seguir por aca, agregar las edades que faltan..........

# Funciones de utilidad

In [ ]:
# Generador de sample
# el id tiene que estar en la columna 0
# fijarse de balancear el sample con 50% de postulados y no postulados
def generador_de_sample(df_1, df_2, n_sample, ciclos):
    df_1 = df_1.drop(columns=df_1.columns[1:])
    df_2 = df_2.drop(columns=df_2.columns[1:])
    df_1['tem'] = 1
    df_2['tem'] = 1
    sample_append = pd.DataFrame(columns=[df_1.columns[0],'tem',df_2.columns[0]])
    for i in range(0,ciclos):
        sample_df_1 = df_1.sample(n=n_sample,random_state=i)
        sample_df_2 = df_2.sample(n=n_sample,random_state=i)
        sample_merge = sample_df_1.merge(sample_df_2)
        sample_append = sample_append.append(sample_merge)
    sample_append = sample_append.drop(columns='tem')
    sample_append = sample_append.reset_index().drop(columns='index')
    return sample_append

# Funciones feature

In [ ]:
#duelve un DF booleano
def feature_se_postulo_alguna_vez(df_1,df_2):
    idpostulante = df_2['idpostulante'].value_counts().index
    idaviso = df_2['idaviso'].value_counts().index
    feature = df_1['idpostulante'].isin(idpostulante) & df_1['idaviso'].isin(idaviso)
    #feature = feature.iloc[:,0] & feature.iloc[:,1]
    return feature

In [ ]:
sample = generador_de_sample(gente_no_postulada,avisos_detalle,2,2)

In [ ]:
sample.shape

In [ ]:
sample = sample.append(tupla_postulada.head(8))
sample = sample.reset_index(drop=True)

In [ ]:
sample

In [ ]:
sample['f1'] = feature_se_postulo_alguna_vez(sample,postulaciones)

In [ ]:
sample

In [ ]:
sample = sample.merge(postulantes_genero_y_edad.drop_duplicates('idpostulante'),on='idpostulante')

In [ ]:
sample

In [ ]:
sample = sample.drop(columns=['fechanacimiento','idaviso','idpostulante'])

In [ ]:
sample = oneHotEncoding(sample,'sexo')

In [ ]:
sample

In [ ]:
sample.drop(columns='f1')

In [ ]:
trainData, testData, trainLabels, testLabels = train_test_split(sample.drop(columns='f1'), sample['f1'], test_size=0.10)

In [ ]:
model = sklearn.linear_model.Perceptron(penalty='l2', alpha=1, max_iter=200, verbose=9, n_jobs=-1)
model.fit(trainData, trainLabels)

In [ ]:
predicted = clf.predict()

In [ ]:
postulantes_genero_y_edad.shape

In [ ]:
a = postulantes_genero_y_edad[postulantes_genero_y_edad['idpostulante'].isin(test_final['idpostulante'].value_counts().index)]

In [ ]:
a.isna().sum()

In [ ]:
a['sexo'].value_counts()

In [ ]:
postulantes_educacion.drop()